In [45]:
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing
from xgboost import XGBClassifier

In [84]:
train_data = pd.read_csv('./train.csv')

In [85]:
train_data.head(3)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A


대출기간 : 36 months , 60 months   </br>
주택소유상태 : MORTGAGE,RENT,own,ANY </br>
대출목적 : 부채 통합 ,신용 카드,주택 개선 ,기타 ,주요 구매  ,의료 ,자동차,소규모 사업 ,이사 ,휴가, 주택 ,재생 에너지          
     
       

 

In [86]:
label_encoder = preprocessing.LabelEncoder()
label_encoder_target = preprocessing.LabelEncoder()
train_data['대출기간'] = label_encoder.fit_transform(train_data['대출기간'] )
train_data['주택소유상태'] = label_encoder.fit_transform(train_data['주택소유상태'] )
train_data['대출목적'] = label_encoder.fit_transform(train_data['대출목적'] )
train_data['대출등급'] = label_encoder_target.fit_transform(train_data['대출등급'] )

In [87]:
train_data.head(3)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,0,6 years,3,72000000,18.90,15,1,0,0,0.0,0.0,0.0,2
1,TRAIN_00001,14400000,1,10+ years,1,130800000,22.33,21,10,0,373572,234060.0,0.0,0.0,1
2,TRAIN_00002,12000000,0,5 years,1,96000000,8.60,14,1,0,928644,151944.0,0.0,0.0,0


근로기간

In [88]:
train_data['근로기간'].value_counts()

근로기간
10+ years    31585
2 years       8450
< 1 year      7774
3 years       7581
1 year        6249
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
10+years       896
<1 year        370
3               89
1 years         56
Name: count, dtype: int64

In [89]:
train_data['근로기간'] = train_data['근로기간'].str.extract(r'(\d+)')
train_data['근로기간']  =train_data['근로기간'].fillna(0)
train_data['근로기간']  =train_data['근로기간'].astype(int)

main processing

In [90]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [91]:
train_data = train_data.drop(columns='ID')
train_data_target = train_data['대출등급']
train_data = train_data.drop(columns='대출등급')

In [92]:
train_data_x, valid_data_x, train_data_y, valid_data_y= train_test_split(train_data,train_data_target,test_size=0.2, shuffle=True,random_state=34)

## **모델적용**

In [102]:
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

# 데이터 준비 (X_train, y_train 등은 사전에 준비되어야 합니다)
# 예: X_train, X_test, y_train, y_test = train_test_split(...)

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 10000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.1, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1),
        'subsample': trial.suggest_float('subsample', 0.5, 1)
    }

    model = XGBClassifier(**param, random_state=1004)
    model.fit(train_data_x, train_data_y)
    preds = model.predict(valid_data_x)
    accuracy = accuracy_score(valid_data_y, preds)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)  # 예를 들어 100번의 시도

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2024-01-26 15:42:35,871] A new study created in memory with name: no-name-85afd723-70a3-4c16-949b-2c64430d7876
[I 2024-01-26 15:43:51,821] Trial 0 finished with value: 0.8210187444830989 and parameters: {'n_estimators': 1895, 'max_depth': 9, 'min_child_weight': 5, 'gamma': 2.9634919111628437, 'learning_rate': 0.001037860877879621, 'colsample_bytree': 0.711173080961539, 'reg_lambda': 0.3805653523085371, 'reg_alpha': 0.1843307663984247, 'subsample': 0.8421651349374739}. Best is trial 0 with value: 0.8210187444830989.
[I 2024-01-26 15:45:57,722] Trial 1 finished with value: 0.8624539176488915 and parameters: {'n_estimators': 6620, 'max_depth': 17, 'min_child_weight': 1, 'gamma': 0.5501280774828748, 'learning_rate': 0.04168192718589361, 'colsample_bytree': 0.860743102681893, 'reg_lambda': 0.6776527805465952, 'reg_alpha': 0.12134104830613444, 'subsample': 0.6604438322006899}. Best is trial 1 with value: 0.8624539176488915.
[I 2024-01-26 15:46:58,704] Trial 2 finished with value: 0.820551

Number of finished trials: 100
Best trial: {'n_estimators': 9022, 'max_depth': 19, 'min_child_weight': 3, 'gamma': 0.6495374150423905, 'learning_rate': 0.05241792993352655, 'colsample_bytree': 0.6049988077096885, 'reg_lambda': 0.059608662299264675, 'reg_alpha': 0.238435325461366, 'subsample': 0.9838022692563583}


In [ ]:

optuna.visualization.plot_param_importances(study) # 파라미터 중요도 확인 그래프
optuna.visualization.plot_optimization_history(study) # 최적화 과정 시각화

In [74]:

model = XGBClassifier(random_state=1004, n_estimators=5530, max_depth=16, min_child_weight=2, gamma=1.4712048790022367, learning_rate= 0.061618288246920457,
                      colsample_bytree= 0.5672824979612345, reg_lambda=0.27002323553522906, reg_alpha=0.4911682343686033, subsample=0.8733973195049249)

## **valid set도 train에 포함**

In [103]:
model = XGBClassifier(random_state = 1004, n_estimators = 9022, max_depth = 19, min_child_weight = 3, gamma = 0.6495374150423905, learning_rate = 0.05241792993352655,
                      colsample_bytree = 0.6049988077096885, reg_lambda= 0.059608662299264675, reg_alpha=0.238435325461366, subsample=0.9838022692563583)
model.fit(train_data, train_data_target)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6049988077096885, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.6495374150423905,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05241792993352655,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=19, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=9022, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

## **test**

In [104]:
test_data = pd.read_csv('./test.csv')
test_data.head(2)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,TEST_00000,16800000,36 months,8 years,MORTGAGE,132000000,19.64,12,주택 개선,0,394692,146604.0,0.0,0.0
1,TEST_00001,8400000,36 months,5 years,RENT,89971200,15.84,25,부채 통합,0,0,0.0,0.0,0.0


In [105]:
test_data['대출기간'] = label_encoder.fit_transform(test_data['대출기간'] )
test_data['주택소유상태'] = label_encoder.fit_transform(test_data['주택소유상태'] )
test_data['대출목적'] = label_encoder.fit_transform(test_data['대출목적'] )
test_data['근로기간'] = test_data['근로기간'].str.extract(r'(\d+)')
test_data['근로기간']  =test_data['근로기간'].fillna(0)
test_data['근로기간']  =test_data['근로기간'].astype(int)

In [106]:
test_data = test_data.drop(columns='ID')

In [108]:
test_pred = model.predict(test_data)

In [109]:
test_pred_decoded = label_encoder_target.inverse_transform(test_pred)

In [110]:
test_pred_decoded

array(['B', 'C', 'A', ..., 'D', 'C', 'A'], dtype=object)

submissionfile 생성

In [111]:
submission_data = pd.read_csv('./sample_submission.csv')
submission_data['대출등급'] = test_pred_decoded
submission_data.to_csv('./xgboost.csv', index=False)


In [112]:
submission_data

,ID,대출등급
0,TEST_00000,B
1,TEST_00001,C
2,TEST_00002,A
3,TEST_00003,C
4,TEST_00004,C
...,...,...
64192,TEST_64192,D
64193,TEST_64193,D
64194,TEST_64194,D
64195,TEST_64195,C
